In [1]:
import pandas as pd
import numpy as np

# Models
from statsmodels.stats.outliers_influence import variance_inflation_factor
import sklearn.metrics as metrics
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, RidgeCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Plot
import matplotlib.pyplot as plt
import seaborn as sns

# Settings
from varname import nameof
from sklearn.metrics import matthews_corrcoef

np.set_printoptions(precision=3)
pd.set_option('precision', 3)

sns.set()

### Клиенты телекоммуникационной компании 

У вас есть набор данных от некой телекоммуникационной компании. Данные содержат информацию о пользователях компании, их демографических характеристиках, услугах, которыми они пользуются, продолжительности пользования услугами компании, способе оплаты и размере оплаты.

Задача состоит в том, чтобы проанализировать данные и спрогнозировать отток пользователей (выявить людей, которые будут и не будут продлевать свой контракт с компанией).


Gender - пол (male / female) <br>
Senior - пенсионер или нет (1, 0)<br>
Married - состоит ли в браке (Yes, No) <br>
Tenure - сколько месяцев человек является клиентом компании<br>
Phone - подключен ли телефон (Yes, No)<br>
MultiplePhone - подключено ли несколько телефонных каналов (Yes, No, No phone service)<br>
Internet - предоставляются ли услуги интернет соединения (DSL, Fiber optic, No)<br>
Security - подключен ли сервис интернет-безопасности (Yes, No, No internet service)<br>
Backup - активирована ли служба онлайн-резервного копирования (Yes, No, No internet service)<br>
Insurance - есть ли у клиента страховка оборудования (Yes, No, No internet service)<br>
Support - подключена ли служба технической поддержки (Yes, No, No internet service)<br>
TV - подключена ли служба потокового телевидения (Yes, No, No internet service)<br>
Movies - активирован ли сервис потокового кино (Yes, No, No internet service)<br>
Contract - тип клиентского контракта (Month-to-month, One year, Two year)<br>
EBilling - использует ли клиент безбумажный биллинг (Yes, No)<br>
PaymentMethod - способ оплаты (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic))<br>
MonthlyCharges - текущий ежемесячный платеж<br>
TotalCharges - общая сумма, которую клиент заплатил за услуги за все время<br>
Churn - покинул ли клиент компанию (Yes or No)<br>

In [121]:
test = pd.read_csv("test.csv", index_col=0)

In [122]:
data = pd.read_csv('train.csv', index_col=0)

In [123]:
# Поле для экспериментов

In [124]:
data.TotalCharges = data.TotalCharges.replace(' ', 0).astype(float)

In [125]:
test.TotalCharges = test.TotalCharges.replace(' ', 0).astype(float)

In [126]:
data = data.replace({'Gender': {'Female': 1, 'Male': 0}, 
                     #'Married': {'Yes': 1, 'No': 0}, 
                     'Phone': {'Yes': 1, 'No': 0},
                     'EBilling': {'Yes': 1, 'No': 0}, 
                     #'Dependents': {'Yes': 1, 'No': 0}, 
                     #'MultiplePhones': {'Yes': 1, 'No': 0, 'No phone service': 0}, 
                     'Security': {'Yes': 1, 'No': 0, 'No internet service': 0}, 
                     'Backup': {'Yes': 1, 'No': 0, 'No internet service': 0}, 
                     'Insurance': {'Yes': 1, 'No': 0, 'No internet service': 0}, 
                     'Support': {'Yes': 1, 'No': 0, 'No internet service': 0}, 
                     'TV': {'Yes': 1, 'No': 0, 'No internet service': 0}, 
                     'Movies': {'Yes': 1, 'No': 0, 'No internet service': 0}, 
                     'Contract': {'Month-to-month': 0, 'One year': 1, 'Two year': 2}, 
                     'Internet': {'Fiber optic': 1, 'DSL': 2, 'No': 0}, 
                    })

In [127]:
test = test.replace({'Gender': {'Female': 1, 'Male': 0}, 
                     #'Married': {'Yes': 1, 'No': 0}, 
                     'Phone': {'Yes': 1, 'No': 0},
                     'EBilling': {'Yes': 1, 'No': 0}, 
                     #'Dependents': {'Yes': 1, 'No': 0}, 
                     #'MultiplePhones': {'Yes': 1, 'No': 0, 'No phone service': 0}, 
                     'Security': {'Yes': 1, 'No': 0, 'No internet service': 0}, 
                     'Backup': {'Yes': 1, 'No': 0, 'No internet service': 0}, 
                     'Insurance': {'Yes': 1, 'No': 0, 'No internet service': 0}, 
                     'Support': {'Yes': 1, 'No': 0, 'No internet service': 0}, 
                     'TV': {'Yes': 1, 'No': 0, 'No internet service': 0}, 
                     'Movies': {'Yes': 1, 'No': 0, 'No internet service': 0}, 
                     'Contract': {'Month-to-month': 0, 'One year': 1, 'Two year': 2}, 
                     'Internet': {'Fiber optic': 1, 'DSL': 2, 'No': 0}, 
                    })

In [128]:
# Выбираем только числовые столбцы

In [129]:
train_numeric_data = data.select_dtypes(include=np.number)
test_numeric_data = test.select_dtypes(include=np.number)

In [130]:
# Обучаем модели

In [131]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(train_numeric_data, 
                                                    data.Churn,
                                                    test_size = 0.20,
                                                    random_state = 1234)


In [132]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()

dt.fit(X_train, Y_train)

dt_train_predictions = dt.predict(X_test)

matthews_corrcoef(Y_test, dt_train_predictions)

0.27085315589030434

In [133]:
smart_dt = DecisionTreeClassifier(min_samples_leaf = 7, max_depth = 17).fit(X_train, Y_train)
print (f'Tree depth: {smart_dt.get_depth()}')

print('Test evaluation')
sdt_test_predictions = smart_dt.predict (X_test)
matthews_corrcoef(Y_test, sdt_test_predictions)

Tree depth: 15
Test evaluation


0.3747280791291233

In [134]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold

tree_params = {'min_samples_leaf': [5, 7, 9, 11],
               'max_depth': [12, 15, 17, 19],
               'criterion': ['gini', 'entropy']              
              }

grid = GridSearchCV(DecisionTreeClassifier(),
                    tree_params,
                    refit=True,
                    scoring='f1_macro',
                    n_jobs = -1,
                    cv=StratifiedKFold(n_splits = 5),
                    verbose=2
                   )

grid_best_model = grid.fit(X_train, Y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


In [135]:
print('Test evaluation')
gbm_test_predictions = grid_best_model.predict (X_test)
matthews_corrcoef(Y_test, gbm_test_predictions)

Test evaluation


0.40015117535930655

In [136]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

lr = LogisticRegression(solver='liblinear', max_iter=1000)

cv_lr_predictions = cross_val_predict(lr, X_train, Y_train, cv = 5, n_jobs = -1)


lr = lr.fit(X_train, Y_train)
print('Test evaluation')
lr_test_predictions = lr.predict (X_test)
matthews_corrcoef(Y_test, lr_test_predictions)

Test evaluation


0.47458695229183945

In [137]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train, Y_train)

y_pred = knn.predict(X_test)
matthews_corrcoef(Y_test, y_pred)

0.34229639137813295

In [138]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

nb.fit(X_train,Y_train)

Y_pred_nb = nb.predict(X_test)

In [139]:
matthews_corrcoef(Y_test, Y_pred_nb)

0.42392563017301976

In [140]:
# Получаем предсказания для тестовых данных

In [141]:
predictions_test = lr.predict (test_numeric_data)

In [142]:
# Сохраняем данные

In [143]:
predictions_df = pd.DataFrame(data=predictions_test, columns=["Churn"])

In [144]:
predictions_df

,Churn
0,False
1,True
2,False
3,False
4,True
...,...
2390,False
2391,False
2392,True
2393,True


In [26]:
predictions_df.Churn.value_counts()

False    1894
True      501
Name: Churn, dtype: int64

In [146]:
predictions_df.to_csv(index=True, index_label='Index', path_or_buf='sample_submission.csv')

In [145]:
predictions_df.Churn.value_counts()

False    1881
True      514
Name: Churn, dtype: int64